# Wildland Fire Data Analysis

The goal of this notebook is to analysis wildland fires for Indianapolis, Indiana. 

In [1]:
#########
#
#    IMPORTS
#

# In-built libraries
import os
import json

# Third-party libraries
import numpy as np
import pandas as pd
import geopandas as gpd

# Other third-party libraries
import copy
import fiona
from shapely.geometry import Polygon, Point
import contextily as cx
import folium.folium
import matplotlib.pyplot as plt
from geopandas.explore import _categorical_legend
from folium.plugins import Geocoder
from matplotlib.colors import rgb2hex

# Local libraries
from utils import plot_gdf, explore_gdf

In [9]:
#########
#
#    CONSTANTS
#
FIRE_ANNUAL_SMOKE_ESTIMATES_PATH = "final/fire_annual_smoke_estimates.csv"
AQI_ANNUAL_ESTIMATES_PATH = "final/aqi_annual_estimates.csv"

FILTERED_FIRE_FEATURES_PATH = "intermediate/filtered_fire_features.geojson"

DATA_PATH = "data/Fire_Feature/Fire_Feature_Data.gdb"

## Load the Data

In [6]:
filtered_fire_features_gdf = gpd.read_file(FILTERED_FIRE_FEATURES_PATH)

In [10]:
fire_annual_smoke_estimates_df = pd.read_csv(FIRE_ANNUAL_SMOKE_ESTIMATES_PATH)